#### Higher Order Functions

1. Implement `get_longest`, `get_shortest`, `get_oldest`, `get_newest`

In [120]:
songs = [
  { 'name': 'Space Oddity', 'duration': 304, 'released': 1972 },
  { 'name': 'Karma Police', 'duration': 263, 'released': 1997 },
  { 'name': 'Pictures Of You', 'duration': 503, 'released': 1993 },
  { 'name': 'Wonderwall', 'duration': 280, 'released': 1995 },
  { 'name': 'Let It Happen', 'duration': 256, 'released': 2015 },
  { 'name': 'Where Is My Mind', 'duration': 216, 'released': 1988 },
]

def get_longest(songs):
  result = songs[0]
  for song in songs:
    if (result['duration'] < song['duration']):
      result = song
  return result

def get_shortest(songs):
  result = songs[0]
  for song in songs:
    if (result['duration'] > song['duration']):
      result = song
  return result

def get_oldest(songs):
  result = songs[0]
  for song in songs:
    if (result['released'] > song['released']):
      result = song
  return result

def get_newest(songs):
  result = songs[0]
  for song in songs:
    if (result['released'] < song['released']):
      result = song
  return result


print(get_longest(songs))
print(get_shortest(songs))
print(get_oldest(songs))
print(get_newest(songs))

{'name': 'Pictures Of You', 'duration': 503, 'released': 1993}
{'name': 'Where Is My Mind', 'duration': 216, 'released': 1988}
{'name': 'Space Oddity', 'duration': 304, 'released': 1972}
{'name': 'Let It Happen', 'duration': 256, 'released': 2015}


2. Implement `my_reduce`, `my_reduce_with_predicate` and rewrite solution above.

In [121]:
def my_reduce(reducer, initial, arr):
  acc = initial
  for item in arr:
    acc = reducer(acc, item)
  return acc

def my_reduce_with_predicate(predicate, arr):
  return my_reduce(lambda acc, item: acc if predicate(acc, item) else item, arr[0], arr)

def fn_longest(songs):
  return my_reduce_with_predicate(lambda a, b: a['duration'] > b['duration'], songs)

def fn_shortest(songs):
  return my_reduce_with_predicate(lambda a, b: a['duration'] < b['duration'], songs)

def fn_oldest(songs):
  return my_reduce_with_predicate(lambda a, b: a['released'] < b['released'], songs)

def fn_newest(songs):
  return my_reduce_with_predicate(lambda a, b: a['released'] > b['released'], songs)


print(fn_longest(songs))
print(fn_shortest(songs))
print(fn_oldest(songs))
print(fn_newest(songs))

{'name': 'Pictures Of You', 'duration': 503, 'released': 1993}
{'name': 'Where Is My Mind', 'duration': 216, 'released': 1988}
{'name': 'Space Oddity', 'duration': 304, 'released': 1972}
{'name': 'Let It Happen', 'duration': 256, 'released': 2015}


3. Power of reduce - `my_map`, `my_filter`, `my_find`

In [122]:
def my_map(mapper, arr):
  return my_reduce(lambda acc, item: [*acc, mapper(item)], [], arr)

def my_filter(predicate, arr):
  return my_reduce(lambda acc, item: [*acc, item] if predicate(item) else acc, [], arr)

def my_find(predicate, arr):
  return my_filter(predicate, arr)[0]


print(my_map(lambda song: song['duration'], songs))
print(my_filter(lambda song: song['duration'] < 270, songs))
print(my_find(lambda song: song['name'] == 'Space Oddity', songs))

[304, 263, 503, 280, 256, 216]
[{'name': 'Karma Police', 'duration': 263, 'released': 1997}, {'name': 'Let It Happen', 'duration': 256, 'released': 2015}, {'name': 'Where Is My Mind', 'duration': 216, 'released': 1988}]
{'name': 'Space Oddity', 'duration': 304, 'released': 1972}


4. Implement `memoize`.

In [123]:
from functools import wraps

def memoize(pure_fn):
  cache = {}
  
  @wraps(pure_fn)
  def memoized(*args):
    str_args = str(args)
    if str_args not in cache:
      cache[str_args] = pure_fn(*args)
    return cache[str_args]

  return memoized

@memoize
def fibonacci(n):
  if (n < 2):
    return 1
  return fibonacci(n - 1) + fibonacci(n - 2)

print(fibonacci(40))

165580141


#### Currying

1. Implement `curry`.

In [124]:
from inspect import signature

def curry(func):
  
  @wraps(func)
  def curried(*args):
    if len(args) >= len(signature(func).parameters):
      return func(*args)
  
    else:
      @wraps(func)
      def partially_applied(*args2):
        return curried(*args, *args2)
      return partially_applied
  
  return curried

@curry
def concat(a, b, c):
  return a + b + c

@curry
def print_with_prefix(prefix, str):
  return print(f'{prefix}{str}')

print(concat('First', 'Second', 'Third'))
print(concat('First', 'Second')('Third'))
print(concat('First')('Second', 'Third'))
print(concat('First')('Second')('Third'))
print_with_prefix('Prefix', 'Str')

FirstSecondThird
FirstSecondThird
FirstSecondThird
FirstSecondThird
PrefixStr


2. Implement curried versions of `add`, `prop`, `equals`, `prop_eq`, `filter`, `map`, `reduce`

In [125]:
add = curry(lambda a, b: a + b)
prop = curry(lambda name, obj: obj[name])
equals = curry(lambda a, b: a == b)
prop_eq = curry(lambda name, expected, obj: equals(expected, prop(name, obj)))
filter = curry(my_filter)
map = curry(my_map)
reduce = curry(my_reduce)

print(add(1)(2))
print(prop('name')(songs[0]))
print(equals(1)(2))
print(prop_eq('name')('Space Oddity')(songs[0]))
print(filter(lambda item: item % 2 == 0, [1, 2, 3, 4, 5]))
print(map(lambda item: item * 2, [1, 2, 3, 4, 5]))
print(reduce(lambda acc, item: acc + item, 0, [1, 2, 3, 4, 5]))

3
Space Oddity
False
True
[2, 4]
[2, 4, 6, 8, 10]
15


Imagine we have a data structure called `MessageQueue`:

In [126]:
class MessageQueue:
  def __init__(self):
    self.subscribers = []
  
  def publish(self, task):
    for subscriber in self.subscribers:
      reduce(lambda task, handler: handler(task), task, subscriber['handlers'])
  
  def subscribe(self):
    new_subscriber = {
      'handlers': []
    }
    self.subscribers.append(new_subscriber)

    class Subscriber:
      def then(self, callback):
        new_subscriber['handlers'].append(callback)
        return self

    return Subscriber()

message_queue = MessageQueue()

You can subscribe to messages pushed on the queue by calling `subscribe` and attaching different handlers with `then`:

In [127]:
message_queue \
  .subscribe() \
  .then((lambda data: print(f'test subscribe: { data["result"] }')))

<__main__.MessageQueue.subscribe.<locals>.Subscriber at 0x194f311b690>

You can put a new message on the queue by calling `publish`: 

In [128]:
message_with_task_list = {
  'result': 'SUCCESS',
  'interfaceVersion': '1.0.3',
  'requested': '10/17/2013 15:31:20',
  'lastUpdated': '10/16/2013 10:52:39',
  'tasks': [
    { 'id': 104, 'complete': False, 'priority': 'high', 'username': 'Scott', 'title': 'Do something', 'estimate': 5 },
    { 'id': 105, 'complete': False, 'priority': 'medium', 'username': 'Lena', 'title': 'Do something else', 'estimate': 2 },
    { 'id': 107, 'complete': True, 'priority': 'high', 'username': 'Mike', 'title': 'Fix the foo', 'estimate': 1 },
    { 'id': 108, 'complete': False, 'priority': 'low', 'username': 'Mike', 'title': 'Adjust the bar', 'estimate': 3 },
    { 'id': 110, 'complete': False, 'priority': 'medium', 'username': 'Scott', 'title': 'Rename everything', 'estimate': 8 },
    { 'id': 110, 'complete': True, 'priority': 'medium', 'username': 'Scott', 'title': 'Deploy', 'estimate': 1 },
    { 'id': 112, 'complete': True, 'priority': 'high', 'username': 'Lena', 'title': 'Approve', 'estimate': 13 },
  ],
}
message_queue.publish(message_with_task_list)

test subscribe: SUCCESS


We would like to have `get_incomplete_tasks_estimation`.
It accepts `username` as parameter, and returns the total estimation of all incomplete tasks assigned to that person.

Implement this function in three different ways:

a) imperative

In [129]:
def get_incomplete_tasks_estimation_imperative(username):
  def get_tasks(data):
    return data['tasks']

  def get_user_tasks(tasks):
    filtered = []
    for task in tasks:
      if task['username'] == username:
        filtered.append(task)
    return filtered

  def get_user_incomplete_tasks(user_tasks):
    filtered = []
    for task in user_tasks:
      if task['complete'] == False:
        filtered.append(task)
    return filtered

  def get_estimates(user_incomplete_tasks):
    mapped = []
    for task in user_incomplete_tasks:
      mapped.append(task['estimate'])
    return mapped

  def get_result(estimates):
    acc = 0
    for estimate in estimates:
      acc += estimate
    return acc
  
  def print_result(result):
    print_with_prefix('imperative: ', result)

  return message_queue.subscribe() \
    .then(get_tasks) \
    .then(get_user_tasks) \
    .then(get_user_incomplete_tasks) \
    .then(get_estimates) \
    .then(get_result) \
    .then(print_result)

get_incomplete_tasks_estimation_imperative('Scott')
message_queue.publish(message_with_task_list)

test subscribe: SUCCESS
imperative: 13


b) functional (using higher order functions, but without currying)

In [130]:
def get_incomplete_tasks_estimation_functional(username):
  return message_queue.subscribe() \
    .then(lambda data: data['tasks']) \
    .then(lambda tasks: my_filter(lambda task: task['username'] == username, tasks)) \
    .then(lambda user_tasks: my_filter(lambda task: task['complete'] == False, user_tasks)) \
    .then(lambda user_incomplete_tasks: my_map(lambda task: task['estimate'], user_incomplete_tasks)) \
    .then(lambda estimates: my_reduce(lambda acc, estimate: acc + estimate, 0, estimates)) \
    .then(lambda result: print_with_prefix('functional (no curry): ', result))

get_incomplete_tasks_estimation_functional('Scott')
message_queue.publish(message_with_task_list)

test subscribe: SUCCESS
imperative: 13
functional (no curry): 13


c) functional (using both higher order functions and currying)

In [131]:
def get_incomplete_tasks_estimation_functional(username):
  return message_queue.subscribe() \
    .then(prop('tasks')) \
    .then(filter(prop_eq('username', username))) \
    .then(filter(prop_eq('complete', False))) \
    .then(map(prop('estimate'))) \
    .then(reduce(add, 0)) \
    .then(print_with_prefix('functional (with curry): '))

get_incomplete_tasks_estimation_functional('Scott')
message_queue.publish(message_with_task_list)

test subscribe: SUCCESS
imperative: 13
functional (no curry): 13
functional (with curry): 13


#### Composition

1. Implement `simple_compose`. Check if it's associative.

In [132]:
def simple_compose(f, g):
  def composition(x):
    return f(g(x))
  return composition

head = lambda x: x[0]
to_upper_case = lambda x: x.upper()
exclaim = lambda x: f'{x}!'

fg_h = simple_compose(simple_compose(exclaim, to_upper_case), head)
f_gh = simple_compose(exclaim, simple_compose(to_upper_case, head))

print(fg_h(['first', 'last']))
print(f_gh(['first', 'last']))

FIRST!
FIRST!


1. Implement `compose`.

In [133]:
def compose(*functions):
  if len(functions) == 1:
    return functions[0]
  elif len(functions) <= 2:
    return simple_compose(*functions)
  head, *tail = functions
  return simple_compose(head, compose(*tail))

fgh = compose(exclaim, to_upper_case, head)
print(fgh(['first', 'last']))

FIRST!


2. Implement `simple_pipe` and `pipe`.

In [134]:
def simple_pipe(f, g):
  def piped(x):
    return g(f(x))
  return piped

def pipe(*functions):
  if len(functions) == 1:
    return functions[0]
  if len(functions) <= 2:
    return simple_pipe(*functions)
  head, *tail = functions
  return simple_pipe(head, pipe(*tail))

hgf = pipe(head, to_upper_case, exclaim)
print(hgf(['first', 'last']))

FIRST!


3. Implement `scott_incomplete_tasks_estimation_compose` and `scott_incomplete_tasks_estimation_pipe`

In [135]:
scott_incomplete_tasks_estimation_compose = compose(
    print_with_prefix('incomplete tasks estimation (compose): '),
    reduce(add, 0),
    map(prop('estimate')),
    filter(prop_eq('complete', False)),
    filter(prop_eq('username', 'Scott')),
    prop('tasks')
)

scott_incomplete_tasks_estimation_compose(message_with_task_list)

scott_incomplete_tasks_estimation_pipe = pipe(
    prop('tasks'),
    filter(prop_eq('username', 'Scott')),
    filter(prop_eq('complete', False)),
    map(prop('estimate')),
    reduce(add, 0),
    print_with_prefix('incomplete tasks estimation (pipe): ')
)

scott_incomplete_tasks_estimation_pipe(message_with_task_list)

incomplete tasks estimation (compose): 13
incomplete tasks estimation (pipe): 13


#### Functors

1. Implement `Maybe` functor.

In [136]:
class Nothing:
  pass

class Maybe:
  __nothingValue = Nothing()
  
  def __init__(self, value):
    self.__value = value

  def __repr__(self):
    return 'Nothing' if self.is_nothing() else f'Just({self.__value})'
  
  @staticmethod
  def Just(value):
    return Maybe(value)
  
  @staticmethod
  def Nothing():
    return Maybe(Maybe.__nothingValue)
  
  @staticmethod
  def From(value):
    return Maybe.Nothing() if value is None else Maybe.Just(value)

  def is_nothing(self):
    return self.__value == Maybe.__nothingValue
  
  def unsafe_unwrap_value(self):
    return self.__value
  
  def get_or_else(self, value):
    return value if self.is_nothing() else self.__value
  
  def map(self, fn):
    return self if self.is_nothing() else Maybe.Just(fn(self.__value))
  
  def chain(self, fn):
    return self if self.is_nothing() else fn(self.__value)

  
print(Maybe.Just(5))
print(Maybe.Nothing())
print(Maybe.From(10))
print(Maybe.From(None))

@curry
def at(index, array):
  return array[index] if index < len(array) else None

head = at(0)
inc = add(1)

print(
  Maybe.From(head([5, 2])) \
    .map(inc) \
    .get_or_else('cannot calculate the value')
)


Just(5)
Nothing
Just(10)
Nothing
6


2. Make `Maybe` a monad.

In [137]:
@curry
def divide(a, b):
  return Maybe.Nothing() if b == 0 else Maybe.Just(a // b)

print(
  Maybe.From(head([5, 2])) \
    .chain(divide(10))
)

Just(2)


3. Implement `get_first_incomplete_task_assignee` (with handling missing values):

In [138]:
example_task = {
  'result': 'SUCCESS',
  'interfaceVersion': '1.0.3',
  'requested': '10/17/2013 15:31:20',
  'lastUpdated': '10/16/2013 10:52:39',
  'tasks': [
    { 'id': 104, 'complete': False, 'priority': 'high', 'username': 'Scott', 'title': 'Do something', 'estimate': 5 },
    { 'id': 105, 'complete': False, 'priority': 'medium', 'username': 'Lena', 'title': 'Do something else', 'estimate': 2 },
    { 'id': 107, 'complete': True, 'priority': 'high', 'username': 'Mike', 'title': 'Fix the foo', 'estimate': 1 },
    { 'id': 108, 'complete': False, 'priority': 'low', 'username': 'Mike', 'title': 'Adjust the bar', 'estimate': 3 },
    { 'id': 110, 'complete': False, 'priority': 'medium', 'username': 'Scott', 'title': 'Rename everything', 'estimate': 8 },
    { 'id': 110, 'complete': True, 'priority': 'medium', 'username': 'Scott', 'title': 'Deploy', 'estimate': 1 },
    { 'id': 112, 'complete': True, 'priority': 'high', 'username': 'Lena', 'title': 'Approve', 'estimate': 13 },
  ],
}

a) imperative

In [139]:
def get_first_incomplete_task_assignee_imperative(message):
  if message.get('tasks') is not None:
    incomplete_tasks = filter(lambda task: task['complete'] == False, message.get('tasks'))
    first_incomplete_task = head(incomplete_tasks)
    if first_incomplete_task is not None:
      username = first_incomplete_task.get('username')
      if username is not None:
        return username
  return None

print(get_first_incomplete_task_assignee_imperative(example_task))

def get_first_incomplete_task_assignee_imperative_2(message):
  if message.get('tasks') is None:
    return None
  incomplete_tasks = filter(lambda task: task['complete'] == False, message.get('tasks'))
  first_incomplete_task = head(incomplete_tasks)
  if first_incomplete_task is None:
    return None
  username = first_incomplete_task.get('username')
  if username is None:
    return None
  return username

print(get_first_incomplete_task_assignee_imperative_2(example_task))

Scott
Scott


b) functional (using `Maybe` monad to handle absence of value)

In [140]:
safe_head = curry(lambda arr: Maybe.From(head(arr)))
safe_prop = curry(lambda prop, obj: Maybe.From(obj.get(prop)))
map = curry(lambda fn, functor: functor.map(fn))
chain = curry(lambda fn, monad: monad.chain(fn))
get_or_else = curry(lambda value, maybe: maybe.get_or_else(value))

get_first_incomplete_task_assignee_functional = pipe(
  safe_prop('tasks'),
  map(filter(prop_eq('complete', False))),
  chain(safe_head),
  chain(safe_prop('username')),
  get_or_else(None)
)

print(get_first_incomplete_task_assignee_functional(example_task))

Scott
